In [1]:
import pandas as pd
pd.set_option('display.max_colwidth',100)

In [2]:
Data=pd.read_excel('drive/My Drive/Ankit/october2020 codalab/english/Constraint_English_Train.xlsx')
Data.dropna(inplace = True)
Data.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In general the discrepancies in death counts between dif...,real
1,2,States reported 1121 deaths a small rise from last Tuesday. Southern states reported 640 of thos...,real
2,3,Politically Correct Woman (Almost) Uses Pandemic as Excuse Not to Reuse Plastic Bag https://t.co...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testing laboratories in India and as on 25th August 2020...,real
4,5,Populous states can generate large case counts but if you look at the new cases per million toda...,real


In [3]:
Data.columns=['id','Post','label']

In [4]:
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
def remove_sw(txt):
    no_sw =" ".join([a for a in txt.split() if a not in stop])
    return no_sw
Data['Post'] = Data['Post'].apply(lambda x: remove_sw(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
#rem0ving emoji

import re

def deEmojify(txt):
    regrex_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',txt)

Data['Post']=Data['Post'].apply(lambda x:deEmojify(x))

In [6]:
# remove URL
def remove_URL(txt):
    url= re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"",txt)

Data['Post']=Data['Post'].apply(lambda x:remove_URL(x))

#removing numbers
Data['Post']=Data['Post'].str.replace(r'\d+','')
Data.head()

,id,Post,label
0,1,The CDC currently reports deaths. In general discrepancies death counts different sources small...,real
1,2,States reported deaths small rise last Tuesday. Southern states reported deaths.,real
2,3,Politically Correct Woman (Almost) Uses Pandemic Excuse Not Reuse Plastic Bag #coronavirus #nas...,fake
3,4,#IndiaFightsCorona: We #COVID testing laboratories India th August tests done : @ProfBhargava...,real
4,5,Populous states generate large case counts look new cases per million today smaller states show...,real


In [7]:
import keras
from keras.preprocessing.text import Tokenizer
tok1 = Tokenizer(char_level=False, filters ='!"$%&@()*+,-./:;”“<=>?[\\]^_`{|}~\t\n', lower = True)

In [8]:
tok1.fit_on_texts(Data['Post'])
words=len(tok1.word_counts)
words

13718

In [9]:
word_list=[tok1.word_index]
#word_list

In [10]:
encoded=tok1.texts_to_sequences(Data['Post'])
print(Data['Post'][0])
encoded[0]

The CDC currently reports  deaths. In general discrepancies death counts different sources small explicable. The death toll stands roughly  people today.


[6,
 91,
 223,
 209,
 10,
 47,
 447,
 6628,
 88,
 1166,
 389,
 1645,
 650,
 6629,
 6,
 88,
 615,
 513,
 1646,
 7,
 22]

In [11]:
import numpy as np
from keras.utils.np_utils import to_categorical
classes_list = ["real","fake"]
label_index = Data['label'].apply(classes_list.index)
label1 = np.asarray(label_index)
label = to_categorical(np.asarray(label1))
label

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [12]:
##         Padding encoded sequence of words
from keras.preprocessing import sequence
max_length=20
padd = sequence.pad_sequences(encoded, maxlen=max_length, padding='post')
padd

array([[  91,  223,  209, ..., 1646,    7,   22],
       [  11,   17,   10, ...,    0,    0,    0],
       [4721, 1776,  390, ...,    0,    0,    0],
       ...,
       [  12,  224,  109, ...,    0,    0,    0],
       [2786,  983, 2663, ...,    0,    0,    0],
       [  83,   51,   28, ...,   69,  148,  158]], dtype=int32)

In [13]:
# importing libraries for creating model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Embedding,MaxPool1D
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D,LSTM,Bidirectional

#LSTM

In [25]:
model = Sequential()
model.add(Embedding(input_dim = words + 1, output_dim = 100,input_length=max_length))
#model.add(Conv1D(filters =32,kernel_size=2, activation='relu'))
#model.add(MaxPooling1D(5, strides=1,padding='same'))
model.add(Bidirectional(LSTM(16,return_sequences=True)))
model.add(Bidirectional(LSTM(8,return_sequences=False)))
model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
#model.add(Dense(16, activation = 'relu'))
model.add(Dense(2, activation = 'sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 100)           1371900   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 20, 32)            14976     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 16)                2624      
_________________________________________________________________
flatten_1 (Flatten)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               4352      
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 514       
Total params: 1,394,366
Trainable params: 1,394,366
Non-trainable params: 0
____________________________________________

In [26]:
## compile

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [27]:
model.fit(padd,label,epochs=25,verbose=1,batch_size=64)

Epoch 1/25
101/101 [==============================] - 4s 40ms/step - loss: 0.3878 - acc: 0.8427
Epoch 2/25
101/101 [==============================] - 4s 40ms/step - loss: 0.0833 - acc: 0.9698
Epoch 3/25
101/101 [==============================] - 4s 41ms/step - loss: 0.0220 - acc: 0.9933
Epoch 4/25
101/101 [==============================] - 4s 41ms/step - loss: 0.0062 - acc: 0.9981
Epoch 5/25
101/101 [==============================] - 4s 42ms/step - loss: 0.0020 - acc: 0.9994
Epoch 6/25
101/101 [==============================] - 4s 41ms/step - loss: 2.5789e-04 - acc: 1.0000
Epoch 7/25
101/101 [==============================] - 4s 40ms/step - loss: 9.5686e-05 - acc: 1.0000
Epoch 8/25
101/101 [==============================] - 4s 40ms/step - loss: 5.7220e-05 - acc: 1.0000
Epoch 9/25
101/101 [==============================] - 4s 40ms/step - loss: 4.2136e-05 - acc: 1.0000
Epoch 10/25
101/101 [==============================] - 4s 41ms/step - loss: 3.2369e-05 - acc: 1.0000
Epoch 11/25
101/101

In [28]:
#Accuracy of the model
from sklearn.metrics import classification_report
#print(classification_report(y_test,predictions_test1))

#TESTING

In [29]:
Test =pd.read_excel('drive/My Drive/Ankit/october2020 codalab/english/Constraint_English_Val.xlsx')
Test.dropna(inplace = True)
Test.head()

,id,tweet,label
0,1,Chinese converting to Islam after realising that no muslim was affected by #Coronavirus #COVD19 ...,fake
1,2,11 out of 13 people (from the Diamond Princess Cruise ship) who had intially tested negative in ...,fake
2,3,"COVID-19 Is Caused By A Bacterium, Not Virus And Can Be Treated With Aspirin",fake
3,4,Mike Pence in RNC speech praises Donald Trump’s COVID-19 “seamless” partnership with governors a...,fake
4,5,6/10 Sky's @EdConwaySky explains the latest #COVID19 data and government announcement. Get more ...,real


In [30]:
## doing preprocessing as above

Test['tweet'] = Test['tweet'].apply(lambda x: remove_sw(x))
Test['tweet']=Test['tweet'].apply(lambda x:deEmojify(x))
Test['tweet']=Test['tweet'].apply(lambda x: remove_URL(x))
Test['tweet']=Test['tweet'].str.replace(r'\d+','')
t= Test['tweet']

In [31]:
encoded1 =tok1.texts_to_sequences(t)
print(t[0])
encoded1[0]

Chinese converting Islam realising muslim affected #Coronavirus #COVD country


[245, 9376, 4298, 856, 441, 19, 86]

In [32]:
test_label_index = Test['label'].apply(classes_list.index)
test_label1 = np.asarray(test_label_index)
test_label = to_categorical(np.asarray(test_label1))
test_label[0]

array([0., 1.], dtype=float32)

In [33]:
padded = sequence.pad_sequences(encoded1, maxlen=max_length, padding='post')
padded[0]

array([ 245, 9376, 4298,  856,  441,   19,   86,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [34]:
# testing

test_predictions = model.predict(padded)
# Finding out of two output neurons which one has highest probability
# It will return us the predicted class
# It will convert probability into final class 1 and 0
test_predictions1 = np.zeros_like(test_predictions)
test_predictions1[np.arange(len(test_predictions)), test_predictions.argmax(1)] = 1
#print(test_predictions)
#print(test_predictions1)

In [35]:
#Accuracy 
from sklearn.metrics import classification_report
print(classification_report(test_label,test_predictions1))

              precision    recall  f1-score   support

           0       0.93      0.94      0.94      1120
           1       0.94      0.92      0.93      1020

   micro avg       0.93      0.93      0.93      2140
   macro avg       0.93      0.93      0.93      2140
weighted avg       0.93      0.93      0.93      2140
 samples avg       0.93      0.93      0.93      2140

